In [30]:
using DataFrames, CSV
using JuMP, Gurobi
using LinearAlgebra

In [37]:
distance_raw = Matrix(CSV.read("inputs/distance_matrix.csv", DataFrame, header = true))
demand_raw = Matrix(CSV.read("inputs/salem_demand.csv", DataFrame, header = true))
distance = distance_raw[:, 1:14]
demand = demand_raw[:, 9];


In [ ]:
model = Model(Gurobi.Optimizer)
set_optimizer_attribute(model, "OutputFlag", 0) 

M = 1_000_000

@variable(model, x[i=1:50, j=1:15] >= 0) # amount of waste transported
@variable(model, z[j=1:15], Bin) # landfill selected or not

@constraint(model, [i=1:50], sum(x[i, j] for j in 1:15) == q[i])
@constraint(model, [i=1:50, j=1:15], x[i, j] <= M * z[j])
@constraint(model, sum(z[j] for j in 1:15) <= 5)

@objective(model, Min, sum(distance[i, j] * x[i,j] for i=1:50 for j=1:15))

optimize!(model)

optimal_cost = objective_value(model)
optimal_waste_transported = value.(x)
optimal_landfills = value.(z)


println("Optimal landfills: $optimal_landfills")
println("The optimal landfills built are 3, 4, 7, 13, 14")


# printing optimal cost
pretty_optimal_cost = round(optimal_cost)
println("Total distance traveled by the waste: $pretty_optimal_cost")